# Isolation

The notebook includes an implementation of minimax algorithm with alpha-beta pruning and it's application on the game of isolation. 

## Helper Player classes

2 player types to test against locally:

- `RandomPlayer` - chooses a legal move randomly from among the available legal moves
- `HumanPlayer` - allows *YOU* to play against the AI

In [2]:
%load_ext autoreload
%autoreload 2
from random import randint

In [1]:
class RandomPlayer():
    """Player that chooses a move randomly."""
    def move(self, game, legal_moves, time_left):
        if not legal_moves: return (-1,-1)
        return legal_moves[randint(0,len(legal_moves)-1)]

In [3]:
class HumanPlayer():
    """Player that chooses a move according to
    user's input."""
    def move(self, game, legal_moves, time_left):
        print('\t'.join(['[%d] %s'%(i,str(move)) for i,move in enumerate(legal_moves)] ))
        
        valid_choice = False
        while not valid_choice:
            try:
                index = int(raw_input('Select move index:'))
                valid_choice = 0 <= index < len(legal_moves)

                if not valid_choice:
                    print('Illegal move! Try again.')
            
            except ValueError:
                print('Invalid index! Try again.')
        return legal_moves[index]

## Evaluation Functions

These functions will inform the value judgements the AI will make when choosing moves. There are 2 classes:

- `OpenMoveEvalFn` - Scores the number of moves open for your player. All baseline tests will use this function. Mandatory
- `CustomEvalFn`

In [4]:
class OpenMoveEvalFn():

    def score(self, game, maximizing_player_turn=True):
        if maximizing_player_turn:
            return len(game.get_legal_moves())
        return len(game.get_opponent_moves())

In [5]:
class CustomEvalFn():
 
    def score(self, game, maximizing_player_turn=True):
        my_moves = len(game.get_legal_moves())
        oppo_moves = len(game.get_opponent_moves())
        
        pos1 = game.get_last_move_for_player(game.get_active_player())
        pos2 = game.get_last_move_for_player(game.get_inactive_player())
        
        if maximizing_player_turn:   
            return my_moves -  oppo_moves
        return oppo_moves - my_moves
    
    def check_isolation(self, game, pos1, pos2):
        memory = game.get_state()
        memory = [[-1 if memory[i][j] != game.BLANK else 0 for j in range(len(memory[0]))] for i in range(len(memory))]
        memory[pos2[0]][pos2[1]] = 1
        memory[pos1[0]][pos1[1]] = 0
        print(pd.DataFrame(memory))
        return self.__check_isolation_helper(game, pos1, pos2, memory)
    
    def __check_isolation_helper(self, game, pos, pos2, memory):
        print(pos)
        print(pd.DataFrame(memory))
        
        r, c = pos
        if memory[r][c] == 1:
            return True
        elif memory[r][c] == -1:
            return False
        
        memory[r][c] = -1
        
        deltas = [(0,1), (0,-1), (1,1), (1,0), (1, -1), (-1,1), (-1,0), (-1,-1)]
        for delta in deltas:
            dr, dc = delta
            nr, nc = (r + dr, c + dc)
            
            if nr < 0 or nr >= game.height or nc < 0 or nc > game.width or memory[nr][nc] == -1:
                continue
                        
            if self.__check_isolation_helper(game, (nr, nc), pos2, memory):
                memory[r][c] = 1
                return True
        
        return False

### Tests

In [9]:
from isolation import Board

if __name__ == "__main__":
    p1 = RandomPlayer()
    p2 = RandomPlayer()
    sample_board = Board(p1, p2, width=7, height=7)
    # setting up the board as though we've been playing
    sample_board.move_count = 2
    # 1st board = 7 moves
    sample_board.__board_state__ = [
                [0,2,0,0,0,0,0],
                [0,0,0,0,0,0,0],
                [0,0,1,0,0,0,0],
                [0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0]
    ]
    sample_board.__last_player_move__ = {p1: (2,2), p2: (0,1)}

    # player 1 should have 7 moves available,
    # so board gets a score of 7
    h = OpenMoveEvalFn()
    
    print(sample_board.print_board())
    
    print('This board has a score of %s.'%(h.score(sample_board, True)))

  | 2 |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   | 1 |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 

This board has a score of 7.


## CustomPlayer

In [11]:
class CustomPlayer():

    def __init__(self, search_depth=3, eval_fn=CustomEvalFn()):

        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, legal_moves, time_left):

        start = time_left()

        if game.move_count == 0:
            return int(game.width / 2), int(game.height / 2)
        
        best_move, utility = self.alphabeta(game, start, time_left, depth=self.search_depth)
        pre_utility = utility
        cur = time_left()
        
        depth = self.search_depth
        while start - cur <= 400:
            try:
                depth = depth + 1
                res_move, res_utility = self.alphabeta(game, start, time_left, depth=depth)
                if res_utility is not None and res_utility != float("-inf"):
                    utility = 0.7 * res_utility + 0.3 * pre_utility  
                    if utility > 0.7 * pre_utility:
                        best_move = res_move
                    pre_utility = utility
                cur = time_left()
            except Exception as e:
                break
        # print(start - time_left())
        return best_move

    def utility(self, game, maximizing_player=True):
        
        if game.is_winner(self):
            return float("inf")

        if game.is_opponent_winner(self):
            return float("-inf")

        return self.eval_fn.score(game, maximizing_player)

    def minimax(self, game, start, time_left, depth=float("inf"), maximizing_player=True):

        if depth == 0:
            return None, self.utility(game, maximizing_player)
            
        legal_moves = game.get_legal_moves()
        
        best_move = None
        if len(legal_moves) > 0:
            best_move = legal_moves[0]
            
        if maximizing_player:
            best_val = float("-inf")
        else:
            best_val = float("inf")
        
        depth = depth - 1
        for move in legal_moves:
            b = game.forecast_move(move)
            m, v = self.minimax(b, start, time_left, depth, not maximizing_player)
            if (maximizing_player and v > best_val) or ((not maximizing_player) and v < best_val):
                best_move = move
                best_val = v
            
            if start - time_left() > 400:
                return None, None

        return best_move, best_val

    def alphabeta(self, game, start, time_left, depth=float("inf"), alpha=float("-inf"), beta=float("inf"),
                  maximizing_player=True):
        
        if depth == 0:
            return None, self.utility(game, maximizing_player)
        
        legal_moves = game.get_legal_moves()
        
        best_move = None
        if len(legal_moves) > 0:
            best_move = legal_moves[0]
        
        if maximizing_player:
            best_val = float("-inf")
            for move in legal_moves:
                b = game.forecast_move(move)
                m, v = self.alphabeta(b, start, time_left, depth - 1, alpha, beta, False)
                
                if alpha >= beta:
                    break
                
                if v > best_val:
                    best_val = v
                    alpha = v
                    best_move = move
                    
                if start - time_left() > 400:
                    return None, None
        else:
            best_val = float("inf")
            for move in legal_moves:
                b = game.forecast_move(move)
                m, v = self.alphabeta(b, start, time_left, depth - 1, alpha, beta, True)
                
                if beta <= alpha:
                    break
                
                if v < best_val:
                    best_val = v
                    beta = v
                    best_move = move        
                
                if start - time_left() > 400:
                    return None, None
                
        return best_move, best_val

### Tests

In [13]:
if __name__ == "__main__":
    r = CustomPlayer()
    h = RandomPlayer()
    game = Board(r,h)
    game_copy = game.copy()
    
    moves = [[(3, 3), (0, 3)], [(4, 1), (1, 1)], [(5, 3), (2, 3)], [(3, 2), (3, 1)], [(5, 1), (4, 3)],
             [(3, 0), (2, 2)], [(4, 2), (0, 1)], [(2, 1), (1, 3)], [(4, 0), (0, 5)], [(5, 2), (2, 4)],
             [(4, 4), (1, 6)], [(2, 5), (3, 5)]]
    for move in moves:
        game.__apply_move__(move[0])
        game.__apply_move__(move[1])
    #game.__apply_move__((0,0))
    print(game.print_board())
    
    res = r.alphabeta(game, 5000, lambda : 5000, 15)
    print(res)

  | - |   | - |   | - |   | 
  | - |   | - |   |   | - | 
  | - | - | - | - | 1 |   | 
- | - | - | - |   | 2 |   | 
- | - | - | - | - |   |   | 
  | - | - | - |   |   |   | 
  |   |   |   |   |   |   | 

((0, 4), -inf)


In [16]:
if __name__ == "__main__":
    r = CustomPlayer()
    h = RandomPlayer()
    game = Board(h,r)
    #game.__apply_move__((3,3))
    #game.__apply_move__((1,2))
    game_copy = game.copy()
    winner, move_history, termination = game.play_isolation(5000)
    print game_as_text(winner, move_history, termination, game_copy)

0. (0,3)
  |   |   | 1 |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
0. ... (0,2)
  |   | 2 | 1 |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
1. (2,4)
  |   | 2 | - |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   | 1 |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
1. ... (1,4)
  |   | - | - |   |   |   | 
  |   |   |   | 2 |   |   | 
  |   |   |   | 1 |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
2. (3,2)
  |   | - | - |   |   |   | 
  |   |   |   | 2 |   |   | 
  |   |   |   | - |   |   | 
  |   | 1 |   |   |   |   | 
  |   |   |   |   |